# Image gathering
The robot takes pictures in 5 degree increments to familiarize itself with its environment.

In [ ]:
import cozmo
from cozmo.util import degrees
import cv2
import os
from PIL import Image as im 
import numpy as np

Set up the folder in which to store the raw images.

In [ ]:
# File path configuration
parent_dir = r"C:\Users\Dressler\Pictures"
raw_path = os.path.join(parent_dir, "cozmo_raw")

# Only needs to be run once
try:
    os.mkdir(raw_path)
except FileExistsError:
    print("Directory already exists")

Set up the folder in which to store the resized images

In [ ]:
# File path configuration
resized_path = os.path.join(parent_dir, "cozmo_resized")

# Only needs to be run once
try:
    os.mkdir(resized_path)
except FileExistsError:
    print("Directory already exists")

Take the pictures, storing them in the specified directory

In [ ]:
def take_pictures(robot: cozmo.robot.Robot):
  # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True

  deg_step = 5
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(0, 360, deg_step):
    fileName = "image" + str(d) + ".jpeg"
 
    robot.turn_in_place(degrees(deg_step)).wait_for_completed()
    
    latest_image = robot.world.latest_image

    if latest_image is not None:
      print("image = %s" % latest_image)
      file_path = os.path.join(raw_path, fileName)
      latest_image.raw_image.save(file_path, "JPEG", resolution=10)

Method to downscale the images for faster processing later on.

In [ ]:
def resize_saved(width, height):
    deg_step = 5
    for d in range(0, 360, deg_step):
        fileName = "image" + str(d) + ".jpeg"
        path = r"C:\Users\Dressler\Pictures\cozmo_raw"
        source_path = os.path.join(path, fileName)
        
        image = cv2.imread(source_path)
        res = cv2.resize(image, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    
        rImg = im.fromarray(res)
        rPath = r"C:\Users\Dressler\Pictures\cozmo_resized"
        resized_path = os.path.join(rPath, fileName)
        rImg.save(resized_path, "JPEG", resolution=10)

In [ ]:
cozmo.run_program(take_pictures)
#the raw pictures are 360 X 240
#this is a .67% downscale
resize_saved(240, 180)